In [6]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import numpy.linalg as LA
import random
import pandas as pd
from numpy import inf
import torch
from torch import nn
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import torch.optim as optim
import json
import itertools
import math
import pyprind
from random import sample

In [7]:
############################################### DO NOT TOUCH THIS SNIPPET ############################################### 
df1 = pd.read_csv('data/trajectories-0400-0415.csv')
all_index=df1.index.to_numpy()
last_row=list(np.zeros(18))
df1.loc[len(all_index)]=last_row
df1.loc[len(all_index)+1]=last_row
df1.loc[len(all_index)+2]=last_row
df2 = pd.read_csv('data/trajectories-0500-0515.csv')
df3 = pd.read_csv('data/trajectories-0515-0530.csv')

## Adding two empty columns for velocities ##
df1['V_X']=np.nan
df1['V_Y']=np.nan

lane=7
filtered_df=df1.loc[df1['Lane_ID']==lane]
list_mergingvehicles=list(filtered_df.Vehicle_ID.unique())
df1_merging_outliers= [5,210,376,388,409,458,465,743,1005,1097,1133,1215,1408,1724,1812,1909,2387,2438,1778,2911]

#print(list_mergingvehicles)


####### Filtering data #########
    ####### AND ####### 
#### Calculating Velocities ####
dict_interacting_vehicles={}
vehicles_actually_merging=[]
vehicle_row_indices={}
for v_ego in list_mergingvehicles:
    if v_ego not in df1_merging_outliers:
        
        ## Extra data after merging ##
        extra_time_steps=10
        
        ## Finding interacting cars ## 
        ## For now assume that only cars following and preceeding right after it has merged
        ## the ego vehicle interact with the ego vehicle
        df_ego=df1.loc[df1['Vehicle_ID']==v_ego]
        change_index=df_ego.loc[df_ego['Lane_ID']==6].index.to_numpy()[0] # The index at which lane has changed to 6
        change_Glob_time=int(df1.iloc[change_index]['Global_Time'])
        v_preced_ego=int(df1.iloc[change_index]['Preceding'])
        v_follow_ego=int(df1.iloc[change_index]['Following'])
        dict_interacting_vehicles[str(int(v_ego))]=[v_preced_ego,v_ego,v_follow_ego]
        #v_follow_ego_follow=int(df1.loc[(df1['Vehicle_ID']==v_follow_ego) & (df1['Global_Time']==change_Glob_time)]['Following'])    
        #v_preced_ego_preced=int(df1.loc[(df1['Vehicle_ID']==v_preced_ego) & (df1['Global_Time']==change_Glob_time)]['Preceding'])
        #dict_interacting_vehicles[str(v_ego)]=[v_preced_ego_preced,v_preced_ego,v_ego,v_follow_ego,v_follow_ego_follow]

        ## Indices for ego vehicle
        vehicles_actually_merging.append(v_ego)
        temp=df1.loc[(df1['Vehicle_ID']==v_ego) & (df1['Lane_ID']==7)]
        data_index_required_E=list(temp.loc[temp['Vehicle_ID']==v_ego].index.to_numpy())             

        ############################################################
        ## Indices for vehicle preceding ego - Lets call this VPE ##
        temp_v_preced_ego=df1.loc[df1['Vehicle_ID']==v_preced_ego]
        index_v_preced_ego_at_lchange=temp_v_preced_ego.loc[temp_v_preced_ego['Global_Time']==change_Glob_time].index.to_numpy()
        #print(index_v_preced_ego_at_lchange)
        if v_preced_ego!=0:            
            if len(index_v_preced_ego_at_lchange)!=0:
                index_VPE_lchange=index_v_preced_ego_at_lchange[0]        
            else:
                # This is when VPE is identified but there is no data for it
                index_VPE_lchange=len(all_index)+1
                print("zero",index_v_preced_ego_at_lchange,index_VPE_lchange)        
        
        else:
            # When ther is no vehicle i.e VPE = 0
            index_VPE_lchange=len(all_index)+1
            print("zero zero")
        
        
        data_index_required_VPE=[]
        for i in range(len(data_index_required_E)):
            if index_VPE_lchange==len(all_index)+1:
                data_index_required_VPE.append(index_VPE_lchange)    
            else:
                data_index_required_VPE.append(index_VPE_lchange-i-1)
        
        #print(data_index_required_VPE)
        
        temp_extra_time=[]
        
        for i in range(extra_time_steps):
            if index_VPE_lchange==len(all_index)+1:
                temp_extra_time.append(index_VPE_lchange)    
            else:
                temp_extra_time.append(index_VPE_lchange+i)

        data_index_required_VPE.extend(temp_extra_time)
        data_index_required_VPE.sort()
        
        ############################################################
        
        ############################################################
        ## Indices for vehicle following ego - Lets call this VFE ##
        temp_v_follow_ego=df1.loc[df1['Vehicle_ID']==v_follow_ego]
        index_v_follow_ego_at_lchange=temp_v_follow_ego.loc[temp_v_follow_ego['Global_Time']==change_Glob_time].index.to_numpy()
        #print(index_v_preced_ego_at_lchange)
        if v_follow_ego!=0:            
            if len(index_v_follow_ego_at_lchange)!=0:
                index_VFE_lchange=index_v_follow_ego_at_lchange[0]        
            else:
                index_VFE_lchange=len(all_index)+1
                print("zero")#,index_VPE_lchange)        
        else:
            index_VFE_lchange=len(all_index)+1
            print("zero zero")

        data_index_required_VFE=[]
        for i in range(len(data_index_required_E)):
            if index_VFE_lchange==len(all_index)+1:
                data_index_required_VFE.append(index_VFE_lchange)    
            else:
                data_index_required_VFE.append(index_VFE_lchange-i-1)

        temp_extra_time=[]
        
        for i in range(extra_time_steps):
            if index_VFE_lchange==len(all_index)+1:
                temp_extra_time.append(index_VFE_lchange)    
            else:
                temp_extra_time.append(index_VFE_lchange+i)

        data_index_required_VFE.extend(temp_extra_time)
        data_index_required_VFE.sort()
        ############################################################
        
        # Adding extra time steps after merging - for EGO vehicle
        temp_extra_time_ego=[]
        for i in range(extra_time_steps):
            temp_extra_time_ego.append(data_index_required_E[len(data_index_required_E)-1]+i)
        data_index_required_E.extend(temp_extra_time_ego)
        
        key=str(int(v_ego))
        vehicle_row_indices[key]=[data_index_required_VPE,data_index_required_E,data_index_required_VFE]

        # Velocity - v_ego#

        for row_index in data_index_required_E[1:]:
                vel_X=0
                vel_Y=0                
                dt=0.1 # Sec
                                
                vel_X=(df1.iat[row_index,4]-df1.iat[row_index-1,4])/dt
                df1.iat[row_index,18]=round(vel_X,3)
                
                vel_y=(df1.iat[row_index,5]-df1.iat[row_index-1,5])/dt
                df1.iat[row_index,19]=round(vel_y,3)  
    
        # Velocity - v_preced_ego VPE#

        for row_index in data_index_required_VPE[1:]:
                vel_X=0
                vel_Y=0                
                dt=0.1 # Sec
                                
                vel_X=(df1.iat[row_index,4]-df1.iat[row_index-1,4])/dt
                df1.iat[row_index,18]=round(vel_X,3)
                
                vel_y=(df1.iat[row_index,5]-df1.iat[row_index-1,5])/dt
                df1.iat[row_index,19]=round(vel_y,3)  
        
        # Velocity - v_follow_ego VFE#

        for row_index in data_index_required_VFE[1:]:
                vel_X=0
                vel_Y=0                
                dt=0.1 # Sec
                                
                vel_X=(df1.iat[row_index,4]-df1.iat[row_index-1,4])/dt
                df1.iat[row_index,18]=round(vel_X,3)
                
                vel_y=(df1.iat[row_index,5]-df1.iat[row_index-1,5])/dt
                df1.iat[row_index,19]=round(vel_y,3)  
    

# Train test split  
vehicles_for_training=[]
vehicles_for_testing=[]

index=0
for vehicle in vehicles_actually_merging:
    if vehicle not in vehicles_for_training:
        if index%10==0:
            vehicles_for_testing.append(vehicle)
            index+=1
        else:
            vehicles_for_training.append(vehicle)
            index+=1


dict_taining_data={}
for vehicle in vehicles_for_training:
    key=str(int(vehicle))
    data_index=vehicle_row_indices[key]
    ref_trajectory=[]

    for index_VPE,index_E,index_VFE in zip(data_index[0][1:],data_index[1][1:],data_index[2][1:]):
        temp_traj_point=[df1.iat[index_VPE,4],df1.iat[index_VPE,5],df1.iat[index_VPE,18],df1.iat[index_VPE,19]]
        temp_traj_point.extend([df1.iat[index_E,4],df1.iat[index_E,5],df1.iat[index_E,18],df1.iat[index_E,19]])
        temp_traj_point.extend([df1.iat[index_VFE,4],df1.iat[index_VFE,5],df1.iat[index_VFE,18],df1.iat[index_VFE,19]])
        ref_trajectory.append(temp_traj_point)
    dict_taining_data[key]=ref_trajectory    

dict_test_data={}
for vehicle in vehicles_for_testing:
    key=str(int(vehicle))
    data_index=vehicle_row_indices[key]
    ref_trajectory=[]

    for index_VPE,index_E,index_VFE in zip(data_index[0][1:],data_index[1][1:],data_index[2][1:]):
        temp_traj_point=[df1.iat[index_VPE,4],df1.iat[index_VPE,5],df1.iat[index_VPE,18],df1.iat[index_VPE,19]]
        temp_traj_point.extend([df1.iat[index_E,4],df1.iat[index_E,5],df1.iat[index_E,18],df1.iat[index_E,19]])
        temp_traj_point.extend([df1.iat[index_VFE,4],df1.iat[index_VFE,5],df1.iat[index_VFE,18],df1.iat[index_VFE,19]])
        ref_trajectory.append(temp_traj_point)
    dict_test_data[key]=ref_trajectory   


zero zero
zero zero
zero
zero [] 1048576
zero
zero [] 1048576
zero


In [ ]:
dict_test_data

In [8]:

#########################################################################
################################ AIS_gen ################################

################################ Model 1 ################################ 

class GenerateAis1(nn.Module):

    #def __init__(self,n_input,n_state,n_psi2_in=64,n_psi2_out=128):
    def __init__(self,n_input,n_state,n_psi2_in=8,n_psi2_out=16):
        super(GenerateAis1,self).__init__()
        self.PSI_layer1=nn.Linear(n_input,n_psi2_in)     # Use RELU after
        self.PSI_layer2=nn.Linear(n_psi2_in,n_psi2_out)      # Use RELU after
        self.PSI_layer3=nn.GRUCell(n_psi2_out,n_state)       # This is the Gated layer

    def forward(self,x,h):
        #x=torch.transpose(x,0,1)
        #h=torch.transpose(h,0,1)
        x=torch.relu(self.PSI_layer1(x))
        x=torch.relu(self.PSI_layer2(x))
        h=self.PSI_layer3(x,h)
        return h 

In [9]:
#########################################################################
################################ AIS_pred ###############################

################################ Model 1 ################################ 

class PredictAis1(nn.Module):
    
    #def __init__(self,n_output,n_state,n_phi2_in=16,n_phi2_out=8):
    def __init__(self,n_output,n_state,n_phi2_in=32,n_phi2_out=64):
        super(PredictAis1,self).__init__()
        self.PHI_layer1=nn.Linear(n_state,n_phi2_in)  # Use RELU after
        self.PHI_layer2=nn.Linear(n_phi2_in,n_phi2_out)     # Use RELU after
        self.PHI_layer3=nn.Linear(n_phi2_out,n_output)         # mean vector of a unit-variance multivariate Gaussian distribution, samples from which are used to predict the next observation

    def forward(self,x):
        x=torch.relu(self.PHI_layer1(x))
        x=torch.relu(self.PHI_layer2(x))
        output=self.PHI_layer3(x)
        return output



In [11]:
class NN_structure(object):
    def __init__(self,n_train,n_test,n_input,n_output,n_state,learning_rate,ais_gen_model,ais_pred_model,device):
        self.n_train=n_train
        self.n_test=n_test
        self.n_input=n_input
        self.n_output=n_output
        self.n_state=n_state    # Hidden state size in RNN
        self.learning_rate=learning_rate                
        self.device=device
        self.ais_gen_model=ais_gen_model
        self.ais_pred_model=ais_pred_model
        self.n_psi2_in=8
        self.n_psi2_out=16
        self.n_phi2_in=32
        self.n_phi2_out=64
        
        if ais_gen_model==1:
            self.gen_model=GenerateAis1(self.n_input,self.n_state,self.n_psi2_in,self.n_psi2_out).to(self.device)      
        
        if ais_pred_model==1:
            self.pred_model=PredictAis1(self.n_output,self.n_state,self.n_phi2_in,self.n_phi2_out).to(self.device)



    def save_model_weights(self,text="_"):
        name_gen_path="trained_models/ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_train)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state)+text+".pth"
        torch.save(self.gen_model.state_dict(),name_gen_path)

        name_pred_path="trained_models/ais_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_epochs"+str(self.n_train)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state)+text+".pth"
        torch.save(self.pred_model.state_dict(),name_pred_path)

        return "saved"
    
    def load_model_weights(self,text="_"):
        name_gen_path="trained_models/ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_epochs"+str(self.n_train)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state)+text+".pth"
        #torch.save(self.gen_model.state_dict(),name_gen_path)
        self.gen_model.load_state_dict(torch.load(name_gen_path))
        self.gen_model.eval()


        name_pred_path="trained_models/ais_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_epochs"+str(self.n_train)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state)+text+".pth"
        #torch.save(self.pred_model.state_dict(),name_pred_path)
        self.pred_model.load_state_dict(torch.load(name_pred_path))
        self.pred_model.eval()

        return "loaded"



    def train(self):
        train_bar=pyprind.ProgBar(self.n_train)
        self.optimizer = torch.optim.Adam(list(self.gen_model.parameters()) + list(self.pred_model.parameters()), lr=self.learning_rate, amsgrad=True)
        #loss=nn.MSELoss()
        #torch.distributions.MultivariateNormal(obs_pred_next_probs[0,:], torch.eye(obs_pred_next_probs[0,:].shape[0]).to(device)).log_prob(obs[step+1,:])

        for epochs in range(self.n_train):
            train_bar.update()
            #print("epoch")
            training_car_list=list(dict_taining_data.keys())
            training_car_list=sample(training_car_list,len(training_car_list))# Random order for training

            
            for car in training_car_list:
                trajectory_data=dict_taining_data[str(int(car))]

                # Ais initialization
                ais=0.5*torch.ones(self.n_state,dtype=torch.float).to(self.device)

                car_loss=0                
                    
                for time in range(len(trajectory_data)-1): 
                    origin_offset_VPE=np.array(trajectory_data[0][0:2]) # VPE only for position
                    origin_offset_E=np.array(trajectory_data[0][4:6]) # E only for position
                    origin_offset_VFE=np.array(trajectory_data[0][8:10]) # VFE only for position
                    
                    input_vector=np.array(trajectory_data[time])
                    input_vector[0:2]=input_vector[0:2]-origin_offset_VPE
                    input_vector[4:6]=input_vector[4:6]-origin_offset_E
                    input_vector[8:10]=input_vector[8:10]-origin_offset_VFE

                    ref_vector=np.array(trajectory_data[time+1])
                    ref_vector[0:2]=ref_vector[0:2]-origin_offset_VPE
                    ref_vector[4:6]=ref_vector[4:6]-origin_offset_E
                    ref_vector[8:10]=ref_vector[8:10]-origin_offset_VFE

                    train_input=torch.tensor(input_vector,dtype=torch.float).to(self.device)
                    # Output/Ref is only for the Ego vehicle
                    ref_output=torch.tensor(ref_vector[4:8],dtype=torch.float).to(self.device)

                    ais=self.gen_model(train_input,ais)
                    pred_output=self.pred_model(ais)
                    
                    car_loss+=-torch.distributions.MultivariateNormal(pred_output,torch.eye(pred_output.shape[0]).to(device)).log_prob(ref_output)
                    
                self.optimizer.zero_grad()
                car_loss.backward()
                self.optimizer.step()

        print("saving model weights")
        status=self.save_model_weights()

        return status


    def test(self,vehicle):

        # Testing for only one car data
        # We test predictions based only newest observation
        # Later we will test it only using initial condition

        dict_test_results={}

        for epochs in range(self.n_test):
            test_car=vehicle            
        
            trajectory_data=dict_test_data[str(int(test_car))]

            # Ais initialization
            ais=0.5*torch.ones(self.n_state,dtype=torch.float).to(self.device)

            car_loss=0
            previous_prediction=0

            test_results_trajectory=[]
            for time in range(len(trajectory_data)-1): 

                temp_test_info=[]

                origin_offset_VPE=np.array(trajectory_data[0][0:2]) # VPE only for position
                origin_offset_E=np.array(trajectory_data[0][4:6]) # E only for position
                origin_offset_VFE=np.array(trajectory_data[0][8:10]) # VFE only for position
                
                input_vector=np.array(trajectory_data[time])
                input_vector[0:2]=input_vector[0:2]-origin_offset_VPE
                input_vector[4:6]=input_vector[4:6]-origin_offset_E
                input_vector[8:10]=input_vector[8:10]-origin_offset_VFE

                ref_vector=np.array(trajectory_data[time+1])
                ref_vector[0:2]=ref_vector[0:2]-origin_offset_VPE
                ref_vector[4:6]=ref_vector[4:6]-origin_offset_E
                ref_vector[8:10]=ref_vector[8:10]-origin_offset_VFE

                # Output/Ref is only for the Ego vehicle
                temp_test_info.extend(list(ref_vector[4:8]))

                train_input=torch.tensor(input_vector,dtype=torch.float).to(self.device)
                ref_output=torch.tensor(ref_vector[4:8],dtype=torch.float).to(self.device)

                ais=self.gen_model(train_input,ais)
                pred_output=self.pred_model(ais)

                pred_vector=pred_output.detach().cpu().numpy()

                temp_test_info.extend(list(pred_vector))

                test_results_trajectory.append(temp_test_info)
            
            dict_test_results[str(int(test_car))]=test_results_trajectory


        return dict_test_results




    
    

In [13]:
####### Parameters #######
#n_train=len(sampled_index_all_init)
n_train=1
n_test=1
n_input=12
n_output=4
n_state =24 
learning_rate=0.0003
gen_model=1
pred_model=1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")




In [14]:
####### Initialise the neural network #######

network=NN_structure(n_train,n_test,n_input,n_output,n_state,learning_rate,gen_model,pred_model,device)
print("Training has started")
network.train()


Training has started


0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


saving model weights


'saved'

In [108]:
####### Load the neural network weights from saved models #######

loaded_network=NN_structure(n_train,n_test,n_input,n_output,n_state,learning_rate,gen_model,pred_model,device)
loaded_network.load_model_weights()



'loaded'

In [113]:
test_car=vehicles_for_testing[13]
predicted_outputs=loaded_network.test(test_car)

In [115]:
# Plotting outputs #


x_ref=[]
y_ref=[]
vx_ref=[]
vy_ref=[]
x_pred=[]
y_pred=[]
vx_pred=[]
vy_pred=[]

for elem in predicted_outputs[str(int(test_car))]:
    x_ref.append(-elem[0])
    y_ref.append(elem[1])
    vx_ref.append(elem[2])
    vy_ref.append(elem[3])
    x_pred.append(-elem[4])
    y_pred.append(elem[5])
    vx_pred.append(elem[6])
    vy_pred.append(elem[7])

fig, ax = plt.subplots()
#plt.plot(y_ref,x_ref,'r',linewidth=2.0,label="Actual")
#plt.plot(y_pred,x_pred,'b',linewidth=2.0,label="Predicted")
plt.plot(y_ref,'r',linewidth=2.0,label="Actual")
plt.plot(y_pred,'b',linewidth=2.0,label="Predicted")
plt.savefig('graphs/Comparision_xref_xpred_justonecar_g1_p1_lr0003_g_8_16_p_16_32.png')
plt.close()

In [ ]:
git add .
git branch -M main
git commit -m "adding all files"
git push -u origin main